<a href="https://colab.research.google.com/github/AshnaVirdikar/Internship_files/blob/main/BERT_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ashna'a code for BERT starts here 

Installing ktrain as the main dependency


In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 169kB/s 
     |████████████████████████████████| 6.8MB 28.6MB/s 
     |████████████████████████████████| 983kB 36.5MB/s 
     |████████████████████████████████| 266kB 36.6MB/s 
     |████████████████████████████████| 1.3MB 39.1MB/s 
     |████████████████████████████████| 1.2MB 36.3MB/s 
     |████████████████████████████████| 471kB 39.1MB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
     |████████████████████████████████| 2.9MB 39.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=f502bf0b6ac1c02cf8e1ee03a00e1088ddb6903ba496eb0a1b18d5a8bf731b57
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=1d793bc545e4df02a12069e4842abc10a47cccae6edcdfb8f02571f17ceb6930
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
% pip install transformers

     |████████████████████████████████| 1.1MB 7.8MB/s 
  Found existing installation: sentencepiece 0.1.95
    Uninstalling sentencepiece-0.1.95:
      Successfully uninstalled sentencepiece-0.1.95


Importing libraries

In [ ]:
import os.path
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

Importing our dataset

In [ ]:
# Importing the dataset as pandas df
data=pd.read_csv('Tweet_global_warming.csv')
df = pd.DataFrame(data)
print(data)

                                                  tweet existence  confidence
0     Global warming report urges governments to act...       Yes      1.0000
1     Fighting poverty and global warming in Africa ...       Yes      1.0000
2     Carbon offsets: How a Vatican forest failed to...       Yes      0.8786
3     Carbon offsets: How a Vatican forest failed to...       Yes      1.0000
4     URUGUAY: Tools Needed for Those Most Vulnerabl...       Yes      0.8087
...                                                 ...       ...         ...
6085  @bloodless_coup "The phrase 'global warming' s...         Y      1.0000
6086  Virginia to Investigate Global Warming Scienti...       NaN      1.0000
6087  Global warming you tube parody you will enjoy ...         N      0.6411
6088  One-Eyed Golfer: Don't dare tell me about glob...         N      1.0000
6089  man made global warming a hair brained theory ...         N      1.0000

[6090 rows x 3 columns]


Performing basic analysis on raw data to get basic intuition

In [ ]:
#to check the total number of rows that have duplicate values
df.duplicated().sum() 

127

In [ ]:
# dropping duplicate values 
df=df.drop_duplicates (keep = 'first',inplace=False)
print(df)

                                                  tweet existence  confidence
0     Global warming report urges governments to act...       Yes      1.0000
1     Fighting poverty and global warming in Africa ...       Yes      1.0000
2     Carbon offsets: How a Vatican forest failed to...       Yes      0.8786
3     Carbon offsets: How a Vatican forest failed to...       Yes      1.0000
4     URUGUAY: Tools Needed for Those Most Vulnerabl...       Yes      0.8087
...                                                 ...       ...         ...
6085  @bloodless_coup "The phrase 'global warming' s...         Y      1.0000
6086  Virginia to Investigate Global Warming Scienti...       NaN      1.0000
6087  Global warming you tube parody you will enjoy ...         N      0.6411
6088  One-Eyed Golfer: Don't dare tell me about glob...         N      1.0000
6089  man made global warming a hair brained theory ...         N      1.0000

[5963 rows x 3 columns]


In [ ]:
#Checking for missing values 
df.isnull().sum()

tweet            0
existence     1829
confidence       3
dtype: int64

In [ ]:
#dropping missing values 
df=df.dropna(axis=0,how='any')
df

,tweet,existence,confidence
0,Global warming report urges governments to act...,Yes,1.0000
1,Fighting poverty and global warming in Africa ...,Yes,1.0000
2,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
3,Carbon offsets: How a Vatican forest failed to...,Yes,1.0000
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6084,It's 83廟ﾁ and climbing in NYC. August weather ...,Y,1.0000
6085,"@bloodless_coup ""The phrase 'global warming' s...",Y,1.0000
6087,Global warming you tube parody you will enjoy ...,N,0.6411
6088,One-Eyed Golfer: Don't dare tell me about glob...,N,1.0000


In [ ]:
#count number of nan values in each column
print (df.isnull().sum())

tweet         0
existence     0
confidence    0
dtype: int64


In [ ]:
# to ignore jupytr notebook mein warnings!
import warnings
warnings.filterwarnings('ignore')

Sentiment Analysis

In [ ]:
pip install textblob    #needed for sentiment analysis

In [ ]:
from textblob import TextBlob

df['sentiment']=df['tweet'].apply(lambda Tweet: TextBlob(Tweet).sentiment.polarity)
pd.options.display.max_colwidth=200
print(df.head())

                                                                                                                    tweet  ... sentiment
0  Global warming report urges governments to act|BRUSSELS, Belgium (AP) - The world faces increased hunger and .. [link]  ...      0.00
1                                                                    Fighting poverty and global warming in Africa [link]  ...      0.00
2                                             Carbon offsets: How a Vatican forest failed to reduce global warming [link]  ...     -0.25
3                                             Carbon offsets: How a Vatican forest failed to reduce global warming [link]  ...     -0.25
4                                                URUGUAY: Tools Needed for Those Most Vulnerable to Climate Change [link]  ...      0.00

[5 rows x 4 columns]


In [ ]:
#Finally adding labels 
import numpy as np
conditionList = [
    df['sentiment'] == 0,
    df['sentiment'] > 0,
    df['sentiment'] < 0]
choiceList = ['neutral', 'positive', 'negative']
df['label'] = np.select(conditionList, choiceList, default='no_label')
pd.options.display.max_colwidth=20
df.head()

,tweet,existence,confidence,sentiment,label
0,Global warming r...,Yes,1.0000,0.00,neutral
1,Fighting poverty...,Yes,1.0000,0.00,neutral
2,Carbon offsets: ...,Yes,0.8786,-0.25,negative
3,Carbon offsets: ...,Yes,1.0000,-0.25,negative
4,URUGUAY: Tools N...,Yes,0.8087,0.00,neutral


Splitting the dataset into the Training set and Test set

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x= df.iloc[:, -5].values  #tweets
y = df.iloc[:, -1].values #sentiment column
#using label encoder to change label values to cat values so that log reg can pass them
from sklearn import preprocessing
from sklearn import utils

labelencoder = preprocessing.LabelEncoder()
y =labelencoder.fit_transform(y)
x

array(['Global warming report urges governments to act|BRUSSELS, Belgium (AP) - The world faces increased hunger and .. [link]',
       'Fighting poverty and global warming in Africa [link]',
       'Carbon offsets: How a Vatican forest failed to reduce global warming [link]',
       ...,
       'Global warming you tube parody you will enjoy #IPCC #ocra http://bit.ly/bBGWhC',
       "One-Eyed Golfer: Don't dare tell me about global warming: Twenty-five of the forty-nine golfers making the cut thi... http://bit.ly/akeAxp",
       'man made global warming a hair brained theory 4 the scientifically challenged the IPPC has nothing to back up claims #tcot #p2 #climategate'],
      dtype=object)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)


Part 1 : Preprocessing the data

In [ ]:
#Preprocessing using BERT 
(x_train, y_train), (x_test, y_test), preproc=text.texts_from_array(x_train=x_train,y_train=y_train,
                                                                    x_test=x_test,y_test=y_test,
                                                class_names=['neutral','positive','negative'],
                                                maxlen=350,max_features=1500,
                                              preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


Part 2 : Training the model

In [ ]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


Part 3 : Fitting the model on the data

In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
552/552 [==============================] - 9791s 18s/step - loss: 1.0587 - accuracy: 0.4477 - val_loss: 0.8947 - val_accuracy: 0.5792
Epoch 2/4
552/552 [==============================] - 9773s 18s/step - loss: 0.6891 - accuracy: 0.7275 - val_loss: 0.4840 - val_accuracy: 0.8174
Epoch 3/4
552/552 [==============================] - 9790s 18s/step - loss: 0.2476 - accuracy: 0.9207 - val_loss: 0.4554 - val_accuracy: 0.8561
Epoch 4/4
552/552 [==============================] - 9821s 18s/step - loss: 0.0532 - accuracy: 0.9872 - val_loss: 0.4722 - val_accuracy: 0.8682


Part 4 : Evaluating the model

In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=['neutral','positive','negative' ])            

              precision    recall  f1-score   support

     neutral       0.86      0.77      0.81       196
    positive       0.88      0.90      0.89       359
    negative       0.86      0.90      0.88       272

    accuracy                           0.87       827
   macro avg       0.87      0.85      0.86       827
weighted avg       0.87      0.87      0.87       827



array([[150,  28,  18],
       [ 12, 324,  23],
       [ 13,  15, 244]])

How to Use Our Trained BERT Model
We can call the learner.get_predictor method to obtain a Predictor object capable of making predictions on new raw data.



In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['neutral', 'positive', 'negative']

In [ ]:
predictor.predict(x[:10]) 

['positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral']

Ashna's code ends here